In [1]:
from typing import Union

# Haiku permet aux utilisateurs d'utiliser des modèles de POO tout en permettant un accès complet aux transformations de fonctions pures de JAX pour les reseaux neuronaux.
import haiku as hk
# JAX est concue pour le calcul numérique à haute performance, notamment pour la recherche en apprentissage automatique.
import jax
import jax.numpy as jnp
# NumPy ajoute un support pour les grands tableaux multidimensionnels et les matrices, ainsi qu'une grande collection de fonctions mathématiques pour opérer sur ces tableaux.
import numpy as np
# Pickle est un module permettant la (dé)sérialisation
import pickle

import perceiver
import position_encoding
import io_processors
import bytes_tokenizer
import random_masked_language as rml

In [2]:
#@title Model config
D_MODEL = 768
D_LATENTS = 1280
MAX_SEQ_LEN = 2048

encoder_config = dict(
    num_self_attends_per_block=26,
    num_blocks=1,
    z_index_dim=256,
    num_z_channels=D_LATENTS,
    num_self_attend_heads=8,
    num_cross_attend_heads=8,
    qk_channels=8 * 32,
    v_channels=D_LATENTS,
    use_query_residual=True,
    cross_attend_widening_factor=1,
    self_attend_widening_factor=1)

decoder_config = dict(
    output_num_channels=D_LATENTS,
    position_encoding_type='trainable',
    output_index_dims=MAX_SEQ_LEN,
    num_z_channels=D_LATENTS,
    qk_channels=8 * 32,
    v_channels=D_MODEL,
    num_heads=8,
    final_project=False,
    use_query_residual=False,
    trainable_position_encoding_kwargs=dict(num_channels=D_MODEL))

# The tokenizer is just UTF-8 encoding (with an offset)
tokenizer = bytes_tokenizer.BytesTokenizer()

In [3]:
# Agrandit les tableaux inputs et input_mask pour qu'ils aient
# une taille de max_sequence_length, et les remplit avec des 0
def pad(max_sequence_length: int, inputs, input_mask):
  input_len = inputs.shape[1]
  assert input_len <= max_sequence_length
  pad_len = max_sequence_length - input_len
  padded_inputs = np.pad(
      inputs,
      pad_width=((0, 0), (0, pad_len)),
      constant_values=tokenizer.pad_token)
  padded_mask = np.pad(
      input_mask,
      pad_width=((0, 0), (0, pad_len)),
      constant_values=0)
  return padded_inputs, padded_mask

In [4]:
#@title Decoding Perceiver Model
def apply_perceiver(
    inputs: jnp.ndarray, input_mask: jnp.ndarray) -> jnp.ndarray:
  """Runs a forward pass on the Perceiver.

  Args:
    inputs: input bytes, an int array of shape [B, T]
    input_mask: Array of shape indicating which entries are valid and which are
      masked. A truthy value indicates that the entry is valid.

  Returns:
    The output logits, an array of shape [B, T, vocab_size].
  """
  assert inputs.shape[1] == MAX_SEQ_LEN
  # EXCEPTION A GERER : LE TEXTE EST TROP LONG, IL FAUT LE SEPARER EN DES ENSEMBLES DE
  # PHRASES DE MOINS DE MAX_SEQ_LEN CARACTRES

  # https://gdcoder.com/what-is-an-embedding-layer/
  # Une couche d'integration convertie l'input en un ensemble de vecteurs
  # d'integration dont les tailles sont optimisees pour le calcul

  # Creation de la couche d'integration selon le nombre de mots distincts dans
  # le training set et la dimension voulue des vecteurs d'integrations
  embedding_layer = hk.Embed(
      vocab_size=tokenizer.vocab_size,
      embed_dim=D_MODEL)
  # Conversion de l'input pour rentrer dans la 1ere couche (couche d'integration)
  embedded_inputs = embedding_layer(inputs)

  # Taille de la conversion de l'input
  batch_size = embedded_inputs.shape[0]

  # Parametres de construction des informations entrainables a propos de
  # la position des mots dans la phrase
  input_pos_encoding = perceiver.position_encoding.TrainablePositionEncoding(
      index_dim=MAX_SEQ_LEN, num_channels=D_MODEL)
  #print(input_pos_encoding(batch_size))
  # Ajout des informations de position des mots a l'input
  embedded_inputs = embedded_inputs + input_pos_encoding(batch_size)
  # Initialisation du PerceiverIO
  perceiver_mod = perceiver.Perceiver(
      encoder=perceiver.PerceiverEncoder(**encoder_config),
      decoder=perceiver.BasicDecoder(**decoder_config))
  # Stockage dans output du resultat de l'execution du PerceiverIO a partir de la 1ere couche
  output_embeddings = perceiver_mod(
      embedded_inputs, is_training=False, input_mask=input_mask, query_mask=input_mask)

  # Redimensionnement et decodage de l'output
  logits = io_processors.EmbeddingDecoder(
      embedding_matrix=embedding_layer.embeddings)(output_embeddings)
  return logits

# La transformation de la fonction lui permettra plus tard de lui
# passer des parametres pre-enregistres
# input_pos_encoding sera notamment ecrase par celui pre-enregistre
apply_perceiver = hk.transform(apply_perceiver).apply

In [5]:
# On deserialise les valeurs des hyperparametres du modele.
with open("language_perceiver_io_bytes.pickle", "rb") as f:
  params = pickle.loads(f.read())

In [6]:
# On prend un texte pour les tests
with open('exampleText.txt', 'r') as f:
    inputs_str = f.read()

#pre-traitement des donnees
inputs_str = inputs_str.replace('\n', ' ')
inputs_str = inputs_str.replace('  ', ' ')
inputs_str = inputs_str.replace('--a', '')
inputs_str = inputs_str.replace('--', '-')
inputs_str = inputs_str.split(". ")
for sentenceIndex in range(len(inputs_str)):
    if len(inputs_str[sentenceIndex]) > MAX_SEQ_LEN:
        del inputs_str[sentenceIndex]
        --sentenceIndex
    else:
        inputs_str[sentenceIndex] += '.'
print("Number of sentences too long : " + str(len(inputs_str)-1 - sentenceIndex))

dataSize = sentenceIndex+1
data = [
    {
        "input_str":inputs_str[sentenceIndex],
        "maskedwords":None,
        "maskedwordsIndexesInData":None,
        "maskedwordsIndexesInStr":None,
        "input_tokens":None,
        "inputs":None,
        "input_mask":None,
        "out":None,
        "masked_tokens_predictions":None
    } for sentenceIndex in range(dataSize)
]

Number of sentences too long : 0


In [7]:
percentage = 33
for iterator in range(len(data)):
    data[iterator]['maskedwords'], data[iterator]['maskedwordsIndexesInData'] = rml.chooseMaskedWords(data[iterator]['input_str'], percentage)
    data[iterator]['maskedwordsIndexesInStr'] = rml.findIndexes(data[iterator]['input_str'], data[iterator]['maskedwordsIndexesInData'])
    data[iterator]['input_tokens'] = rml.stringWithMaskedWords(data[iterator]['input_str'], data[iterator]['maskedwordsIndexesInStr'])

In [8]:
for iterator in range(len(data)):
    # inputs est le tableau d'entiers avec une dimension supplementaire
    # c'est comme si on mettait le tableau inputs a l'interieur d'un nouveau tableau
    data[iterator]['inputs'] = data[iterator]['input_tokens'][None]

    # input_mask est l'equivalent unitaire de inputs :
    # pour toutes les valeurs d'inputs, on met un 1
    data[iterator]['input_mask'] = np.ones_like(data[iterator]['inputs'])

    data[iterator]['inputs'], data[iterator]['input_mask'] = pad(MAX_SEQ_LEN, data[iterator]['inputs'], data[iterator]['input_mask'])

In [9]:
results = open("analysis_results.txt", "w")

rng = jax.random.PRNGKey(1)  # Unused
#print(params)
for iterator in range(len(data)):
    currentOut = data[iterator]['out'] = apply_perceiver(params, rng=rng, inputs=data[iterator]['inputs'], input_mask=data[iterator]['input_mask'])

    currentMasked_tokens_predictions = data[iterator]['masked_tokens_predictions'] = []
    currentMaskedwordsIndexesInStr = data[iterator]['maskedwordsIndexesInStr']

    even_number_of_masked_words_indexes_in_str = len(currentMaskedwordsIndexesInStr) - len(currentMaskedwordsIndexesInStr) % 2
    for masked_word_index in range(0, even_number_of_masked_words_indexes_in_str, 2):
    # la prediction sous forme d'entiers est constituee des valeurs maximales
    # aux coordonnees du masque sur la 1ere dimension de out
        currentMasked_tokens_predictions.append(
            currentOut[0,currentMaskedwordsIndexesInStr[masked_word_index]:currentMaskedwordsIndexesInStr[masked_word_index+1]].argmax(axis=-1)
        )

    results.write("-"+str(iterator)+"-\n")
    results.write("Local sentence:"+"\n")
    results.write(data[iterator]['input_str']+"\n")
    results.write("Local sentence with masked bytes:"+"\n")
    results.write(tokenizer.to_string(data[iterator]['input_tokens'])+"\n")
    #results.write("Local sentence predicted:"+"\n")
    #results.write(rml.XXXXXXXXX(data[iterator]['input_tokens'], currentMaskedwordsIndexesInStr)+"\n")


    for local_masked_token_prediction in currentMasked_tokens_predictions:
        results.write("\tLocal greedy predictions:"+"\n")
        results.write("\t"+str(local_masked_token_prediction)+"\n") #predictions sous formes de caracteres
        results.write("\tLocal predicted string:"+"\n")
        results.write("\t"+tokenizer.to_string(local_masked_token_prediction)+"\n\n")

    progression = int((iterator+1)/len(data)*100)
    if progression % 5 < 2:
        print("Progression : "+str(progression)+"%")

results.close()

Progression : 1%


ValueError: I/O operation on closed file.